In [4]:
import numpy as np
import pandas as pd #data manipulation
import csv
import os
import pickle

import mediapipe as mp
import cv2
import time
import imutils
from imutils.video import VideoStream

from sklearn.model_selection import train_test_split
#make a machine learning pipeline/system
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler #standardize the data(it makes the data equal)

#classification models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [5]:
# initialize the holistic model
mp_holistic = mp.solutions.holistic

# initialize the hand model
mp_hands = mp.solutions.hands

# initialize the drawing helper
mp_drawing = mp.solutions.drawing_utils

In [3]:
# initialize the webcam
webcam = VideoStream(src=0).start()
time.sleep(2.0)

holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while True:
    frame = webcam.read()
    frame = imutils.resize(frame, width=800)
    #change from BGR to RGB
    RGB_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    #set the flags to False to prevent coping the image data
    RGB_frame.flags.writeable = False
    
    #printing out the results of the detections
    results = holistic.process(RGB_frame)
    #print(results.face_landmarks)
            
    #set this back to True to copy the image data for rendering
    RGB_frame.flags.writeable = True
    #convert back to BGR for rendering
    BGR_frame = cv2.cvtColor(RGB_frame, cv2.COLOR_RGB2BGR)
            
    # Draw face landmarks
    mp_drawing.draw_landmarks(BGR_frame, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(100,100,0), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(0,230,180), thickness=1, circle_radius=1))

    # Draw pose detections
    mp_drawing.draw_landmarks(BGR_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(210,0,0), thickness=2, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(0,220,0), thickness=2, circle_radius=2))
    
    # Right Hand
    mp_drawing.draw_landmarks(BGR_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(43,100,0), thickness=3, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(100,43,0), thickness=2, circle_radius=2))
    
    # Left Hand
    mp_drawing.draw_landmarks(BGR_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(43,100,0), thickness=3, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(100,43,0), thickness=2, circle_radius=2))
    
    cv2.imshow('LIVE!', BGR_frame)
    key = cv2.waitKey(5)
    if key == 27:
        break

webcam.stop()
cv2.destroyAllWindows()

In [4]:
results.pose_landmarks.landmark #pose landmarks array

[x: 0.6274102
y: 0.48893443
z: -1.1924876
visibility: 1.0
, x: 0.6662984
y: 0.41114068
z: -1.1003848
visibility: 1.0
, x: 0.691354
y: 0.41342345
z: -1.100163
visibility: 0.99999976
, x: 0.71674246
y: 0.41544035
z: -1.1001259
visibility: 1.0
, x: 0.59605145
y: 0.4066217
z: -1.1174489
visibility: 1.0
, x: 0.57048285
y: 0.40491465
z: -1.1182835
visibility: 0.9999999
, x: 0.54471
y: 0.40343374
z: -1.1197155
visibility: 1.0
, x: 0.7464473
y: 0.45891702
z: -0.613134
visibility: 0.9999987
, x: 0.5111267
y: 0.4392901
z: -0.70030475
visibility: 0.99999976
, x: 0.6708582
y: 0.58200604
z: -1.013
visibility: 0.99999905
, x: 0.5768615
y: 0.57535446
z: -1.0359496
visibility: 0.99999905
, x: 0.8852141
y: 0.8805595
z: -0.41919038
visibility: 0.9991078
, x: 0.32334745
y: 0.843022
z: -0.5246798
visibility: 0.99983585
, x: 0.98920333
y: 1.3361173
z: -0.33879352
visibility: 0.07981419
, x: 0.16076237
y: 1.2749075
z: -0.5097793
visibility: 0.21147932
, x: 0.9709445
y: 1.698058
z: -0.8451633
visibility: 0.0

In [5]:
results.face_landmarks.landmark #face landmarks array

[x: 0.62142503
y: 0.52193016
z: -0.04730797
, x: 0.62098485
y: 0.47356924
z: -0.06672884
, x: 0.6220021
y: 0.4925867
z: -0.03982798
, x: 0.61269134
y: 0.42786714
z: -0.04175223
, x: 0.62140787
y: 0.4579171
z: -0.06870779
, x: 0.622439
y: 0.4397461
z: -0.060685467
, x: 0.6255282
y: 0.39982414
z: -0.018719235
, x: 0.5378662
y: 0.40106
z: 0.034735464
, x: 0.6274576
y: 0.366396
z: -0.00425478
, x: 0.6283778
y: 0.34652147
z: -0.0029250302
, x: 0.6320534
y: 0.2795375
z: 0.025005339
, x: 0.6212215
y: 0.5296085
z: -0.04678322
, x: 0.6210883
y: 0.53721005
z: -0.043313485
, x: 0.62113947
y: 0.5419012
z: -0.038473047
, x: 0.6211517
y: 0.5492276
z: -0.037407197
, x: 0.6207821
y: 0.55588526
z: -0.040301085
, x: 0.6203735
y: 0.5648487
z: -0.043902334
, x: 0.61999226
y: 0.57471615
z: -0.043152973
, x: 0.619432
y: 0.5919489
z: -0.033163447
, x: 0.6211793
y: 0.48277253
z: -0.061703704
, x: 0.6104461
y: 0.4833108
z: -0.04521958
, x: 0.5077946
y: 0.35057414
z: 0.10795904
, x: 0.57151073
y: 0.4139154
z: 0

# Capture Landmarks and export to csv

In [6]:
# the number of coordinates we are going to loop through
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
num_coords

501

In [7]:
landmarks = ['labels']
for val in range(1, num_coords+1):
    landmarks = landmarks + ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [8]:
# landmarks[-4:]
landmarks

['labels',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 

In [9]:
# write the coordinates to a csv file
with open('body_coords.csv', mode='w', newline='', encoding='utf-8') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [18]:
## Number of label data to export to our csv file
# Victorious
# Wakanda Forever
# Sad
# Neutral
# Happy

label_name = 'Victorious'

# Capture the Label coordinates and export to CSV

In [19]:
# initialize the webcam
webcam = VideoStream(src=0).start()
time.sleep(2.0)

holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while True:
    frame = webcam.read()
    frame = imutils.resize(frame, width=800)
    #change from BGR to RGB
    RGB_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    #set the flags to False to prevent coping the image data
    RGB_frame.flags.writeable = False
    
    #printing out the results of the detections
    results = holistic.process(RGB_frame)
    #print(results.face_landmarks)
            
    #set this back to True to copy the image data for rendering
    RGB_frame.flags.writeable = True
    #convert back to BGR for rendering
    BGR_frame = cv2.cvtColor(RGB_frame, cv2.COLOR_RGB2BGR)
            
    # Draw face landmarks
    mp_drawing.draw_landmarks(BGR_frame, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(100,100,0), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(0,230,180), thickness=1, circle_radius=1))

    # Draw pose detections
    mp_drawing.draw_landmarks(BGR_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(210,0,0), thickness=2, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(0,220,0), thickness=2, circle_radius=2))
    
    # Right Hand
    mp_drawing.draw_landmarks(BGR_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(43,100,0), thickness=3, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(100,43,0), thickness=2, circle_radius=2))
    
    # Left Hand
    mp_drawing.draw_landmarks(BGR_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(43,100,0), thickness=3, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(100,43,0), thickness=2, circle_radius=2))
            
    try:
        # Extract the pose landmarks
        # store the pose landmarks into a variable pose
        pose = results.pose_landmarks.landmark

        # grab these coordinates(x, y, z & visibility) from pose
        # flatten is used to place all coordinates in a single row
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        # store the pose landmarks into a variable face
        face = results.face_landmarks.landmark

        # grab these coordinates(x, y, z & visibility) from face
        # flatten is used to place all coordinates in a single row
        face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
        
        # concate the pose, face rows and hand rows
        row = pose_row+face_row
        # attach the label name as the first in the row
        row.insert(0, label_name)

        # Export the rows to csv
        with open('body_coords.csv', mode='a', newline='', encoding='utf-8') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(row)
    except:
        pass

    cv2.imshow('LIVE!', BGR_frame)
    key = cv2.waitKey(5)
    if key == 27:
        break

webcam.stop()
cv2.destroyAllWindows()

In [2]:
coords = pd.read_csv('body_coords.csv')
coords.tail(10)

,labels,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1192,Victorious,0.438521,0.489970,-0.351705,1.0,0.465007,0.436194,-0.288701,1.0,0.481486,...,0.002258,0.0,0.519119,0.441205,0.011442,0.0,0.525256,0.433274,0.011370,0.0
1193,Victorious,0.446523,0.491231,-0.280074,1.0,0.472841,0.438778,-0.212122,1.0,0.488968,...,0.001281,0.0,0.520261,0.441459,0.010253,0.0,0.526399,0.434180,0.010084,0.0
1194,Victorious,0.465362,0.491675,-0.131352,1.0,0.492262,0.438934,-0.053708,1.0,0.508501,...,0.000289,0.0,0.541059,0.441994,0.009371,0.0,0.547733,0.433279,0.009356,0.0
1195,Victorious,0.476904,0.491430,-0.174299,1.0,0.503001,0.438929,-0.102291,1.0,0.519245,...,-0.002015,0.0,0.540719,0.439801,0.003822,0.0,0.546700,0.432739,0.003416,0.0
1196,Victorious,0.482707,0.491594,-0.120353,1.0,0.510005,0.439842,-0.044329,1.0,0.525924,...,-0.003264,0.0,0.563581,0.443925,0.004039,0.0,0.569826,0.436994,0.003798,0.0
1197,Victorious,0.486718,0.493156,-0.199073,1.0,0.512525,0.442786,-0.126238,1.0,0.528428,...,-0.003127,0.0,0.563900,0.441942,0.005873,0.0,0.570099,0.434941,0.005680,0.0
1198,Victorious,0.484670,0.498666,-0.817357,1.0,0.512950,0.445665,-0.765301,1.0,0.529542,...,-0.003783,0.0,0.573325,0.434306,0.008039,0.0,0.578969,0.427369,0.007914,0.0
1199,Victorious,0.490182,0.499738,-0.695262,1.0,0.518914,0.446046,-0.646380,1.0,0.535807,...,-0.002563,0.0,0.576783,0.433999,0.009755,0.0,0.582825,0.426347,0.009719,0.0
1200,Victorious,0.508462,0.463294,-0.800563,1.0,0.540558,0.399516,-0.758879,1.0,0.560379,...,-0.000353,0.0,0.620691,0.371199,0.018053,0.0,0.626952,0.360876,0.018751,0.0
1201,Victorious,0.522457,0.457643,-0.792602,1.0,0.555421,0.391237,-0.745123,1.0,0.575970,...,-0.000709,0.0,0.620019,0.370047,0.015968,0.0,0.626161,0.360339,0.016441,0.0


In [3]:
coords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Columns: 2005 entries, labels to v501
dtypes: float64(2004), object(1)
memory usage: 18.4+ MB


In [4]:
coords['labels']

0              Sad
1              Sad
2              Sad
3              Sad
4              Sad
           ...    
1197    Victorious
1198    Victorious
1199    Victorious
1200    Victorious
1201    Victorious
Name: labels, Length: 1202, dtype: object

In [5]:
coords[coords['labels']=='Happy']

,labels,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
202,Happy,0.530122,0.465080,-2.787388,1.0,0.568641,0.385262,-2.735798,1.0,0.596959,...,0.001942,0.0,0.623291,0.374530,0.024305,0.0,0.630882,0.364549,0.025465,0.0
203,Happy,0.535547,0.488488,-2.701794,1.0,0.571978,0.406631,-2.650498,1.0,0.600456,...,0.000845,0.0,0.624139,0.388311,0.024866,0.0,0.631727,0.379150,0.025987,0.0
204,Happy,0.533548,0.481356,-2.625869,1.0,0.568105,0.399052,-2.559809,1.0,0.595277,...,0.000113,0.0,0.623648,0.385375,0.024099,0.0,0.631427,0.376409,0.025254,0.0
205,Happy,0.543840,0.481851,-2.766325,1.0,0.576563,0.399747,-2.693891,1.0,0.603871,...,0.000445,0.0,0.624067,0.385380,0.024726,0.0,0.631652,0.376812,0.025810,0.0
206,Happy,0.537141,0.468953,-2.560622,1.0,0.572843,0.394111,-2.492149,1.0,0.600305,...,0.004048,0.0,0.626109,0.383550,0.029183,0.0,0.634011,0.374386,0.030501,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,Happy,0.517820,0.501184,-1.482167,1.0,0.549603,0.414729,-1.394583,1.0,0.574803,...,-0.004976,0.0,0.597460,0.396312,0.018312,0.0,0.604494,0.386211,0.019245,0.0
468,Happy,0.518758,0.493405,-1.501834,1.0,0.550082,0.406479,-1.406476,1.0,0.575205,...,-0.005478,0.0,0.598248,0.394855,0.018335,0.0,0.605160,0.385276,0.019251,0.0
469,Happy,0.515692,0.494964,-1.479587,1.0,0.547548,0.407431,-1.384646,1.0,0.572928,...,-0.004044,0.0,0.594147,0.387569,0.018268,0.0,0.600912,0.378097,0.018977,0.0
470,Happy,0.510007,0.494974,-1.480096,1.0,0.541203,0.407463,-1.389715,1.0,0.566694,...,-0.005143,0.0,0.596774,0.387221,0.017776,0.0,0.603554,0.378213,0.018501,0.0


In [6]:
coords[coords['labels'] == 'Victorious']

,labels,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
954,Victorious,0.545801,0.554038,-0.671674,1.0,0.576543,0.516982,-0.611211,1.0,0.593659,...,0.000202,0.0,0.612873,0.492764,0.013261,0.0,0.617947,0.485273,0.013676,0.0
955,Victorious,0.537667,0.565558,-0.477229,1.0,0.567208,0.525181,-0.414953,1.0,0.584513,...,-0.000307,0.0,0.613825,0.489884,0.012982,0.0,0.618743,0.482948,0.013231,0.0
956,Victorious,0.528432,0.568867,-0.209574,1.0,0.559281,0.528953,-0.143556,1.0,0.576668,...,0.001103,0.0,0.601711,0.496258,0.014795,0.0,0.606726,0.490370,0.015126,0.0
957,Victorious,0.531603,0.562334,0.361813,1.0,0.560099,0.521707,0.457927,1.0,0.577076,...,0.003075,0.0,0.603932,0.496393,0.014906,0.0,0.609136,0.490331,0.015077,0.0
958,Victorious,0.524213,0.561891,0.113359,1.0,0.550890,0.517903,0.199995,1.0,0.566879,...,0.000114,0.0,0.594345,0.498456,0.012866,0.0,0.599445,0.491412,0.013067,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,Victorious,0.486718,0.493156,-0.199073,1.0,0.512525,0.442786,-0.126238,1.0,0.528428,...,-0.003127,0.0,0.563900,0.441942,0.005873,0.0,0.570099,0.434941,0.005680,0.0
1198,Victorious,0.484670,0.498666,-0.817357,1.0,0.512950,0.445665,-0.765301,1.0,0.529542,...,-0.003783,0.0,0.573325,0.434306,0.008039,0.0,0.578969,0.427369,0.007914,0.0
1199,Victorious,0.490182,0.499738,-0.695262,1.0,0.518914,0.446046,-0.646380,1.0,0.535807,...,-0.002563,0.0,0.576783,0.433999,0.009755,0.0,0.582825,0.426347,0.009719,0.0
1200,Victorious,0.508462,0.463294,-0.800563,1.0,0.540558,0.399516,-0.758879,1.0,0.560379,...,-0.000353,0.0,0.620691,0.371199,0.018053,0.0,0.626952,0.360876,0.018751,0.0


In [7]:
coords[coords['labels'] == 'Sad'].head(9)

,labels,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Sad,0.534589,0.571887,-0.885522,1.0,0.559048,0.539598,-0.893571,1.0,0.572091,...,-0.030944,0.0,0.649782,0.503071,-0.011938,0.0,0.656522,0.494819,-0.012609,0.0
1,Sad,0.504073,0.576852,-0.853958,1.0,0.528009,0.543589,-0.864538,1.0,0.540729,...,-0.029673,0.0,0.647749,0.503751,-0.011737,0.0,0.653978,0.497432,-0.012703,0.0
2,Sad,0.547389,0.560564,-0.756654,1.0,0.557909,0.523474,-0.765102,1.0,0.567973,...,-0.031574,0.0,0.648372,0.506333,-0.013374,0.0,0.654605,0.499907,-0.014212,0.0
3,Sad,0.552699,0.565373,-0.856031,1.0,0.562708,0.524396,-0.858279,1.0,0.574198,...,-0.030461,0.0,0.647975,0.507584,-0.011702,0.0,0.654233,0.501043,-0.012442,0.0
4,Sad,0.561208,0.643290,-2.459429,1.0,0.602270,0.536056,-2.449382,1.0,0.632808,...,-0.035593,0.0,0.653219,0.518922,-0.018812,0.0,0.659633,0.510764,-0.019755,0.0
5,Sad,0.555579,0.668574,-2.529172,1.0,0.600285,0.555357,-2.538815,1.0,0.633280,...,-0.037586,0.0,0.653283,0.537748,-0.021938,0.0,0.659632,0.529597,-0.023032,0.0
6,Sad,0.548749,0.682292,-2.993706,1.0,0.596616,0.565893,-3.000965,1.0,0.629464,...,-0.036719,0.0,0.653519,0.539352,-0.019914,0.0,0.659865,0.531234,-0.020877,0.0
7,Sad,0.557728,0.702583,-3.049396,1.0,0.600931,0.582000,-3.062758,1.0,0.633301,...,-0.036942,0.0,0.652641,0.550316,-0.020903,0.0,0.658402,0.542372,-0.021859,0.0
8,Sad,0.552098,0.699134,-2.974888,1.0,0.599052,0.581237,-2.987146,1.0,0.632035,...,-0.036756,0.0,0.651240,0.552128,-0.020687,0.0,0.657324,0.544453,-0.021639,0.0


# Get the Features and Labels

In [8]:
features = coords.drop('labels', axis=1) #features, independent values
features.astype('float')

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0.534589,0.571887,-0.885522,1.0,0.559048,0.539598,-0.893571,1.0,0.572091,0.543174,...,-0.030944,0.0,0.649782,0.503071,-0.011938,0.0,0.656522,0.494819,-0.012609,0.0
1,0.504073,0.576852,-0.853958,1.0,0.528009,0.543589,-0.864538,1.0,0.540729,0.546660,...,-0.029673,0.0,0.647749,0.503751,-0.011737,0.0,0.653978,0.497432,-0.012703,0.0
2,0.547389,0.560564,-0.756654,1.0,0.557909,0.523474,-0.765102,1.0,0.567973,0.522338,...,-0.031574,0.0,0.648372,0.506333,-0.013374,0.0,0.654605,0.499907,-0.014212,0.0
3,0.552699,0.565373,-0.856031,1.0,0.562708,0.524396,-0.858279,1.0,0.574198,0.520885,...,-0.030461,0.0,0.647975,0.507584,-0.011702,0.0,0.654233,0.501043,-0.012442,0.0
4,0.561208,0.643290,-2.459429,1.0,0.602270,0.536056,-2.449382,1.0,0.632808,0.533709,...,-0.035593,0.0,0.653219,0.518922,-0.018812,0.0,0.659633,0.510764,-0.019755,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,0.486718,0.493156,-0.199073,1.0,0.512525,0.442786,-0.126238,1.0,0.528428,0.442769,...,-0.003127,0.0,0.563900,0.441942,0.005873,0.0,0.570099,0.434941,0.005680,0.0
1198,0.484670,0.498666,-0.817357,1.0,0.512950,0.445665,-0.765301,1.0,0.529542,0.445552,...,-0.003783,0.0,0.573325,0.434306,0.008039,0.0,0.578969,0.427369,0.007914,0.0
1199,0.490182,0.499738,-0.695262,1.0,0.518914,0.446046,-0.646380,1.0,0.535807,0.446117,...,-0.002563,0.0,0.576783,0.433999,0.009755,0.0,0.582825,0.426347,0.009719,0.0
1200,0.508462,0.463294,-0.800563,1.0,0.540558,0.399516,-0.758879,1.0,0.560379,0.399116,...,-0.000353,0.0,0.620691,0.371199,0.018053,0.0,0.626952,0.360876,0.018751,0.0


In [9]:
labels = coords['labels'] #target, dependent values
labels

0              Sad
1              Sad
2              Sad
3              Sad
4              Sad
           ...    
1197    Victorious
1198    Victorious
1199    Victorious
1200    Victorious
1201    Victorious
Name: labels, Length: 1202, dtype: object

# Split the Data for Training

In [10]:
x_train, x_test, y_train, y_test = train_test_split(features,labels ,test_size=0.25, random_state=42, stratify=labels)

In [11]:
#make machine learning pipelines in a dictionary
pipelines = {
    'logReg':make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear')),
    'rigClas':make_pipeline(StandardScaler(), RidgeClassifier()),
    'sgdClas':make_pipeline(StandardScaler(), SGDClassifier()),
    'svc':make_pipeline(StandardScaler(), SVC()),
    'gaussNb':make_pipeline(StandardScaler(), GaussianNB()),
    'xgbClas':make_pipeline(StandardScaler(), XGBClassifier()),
    'desTreeClas':make_pipeline(StandardScaler(), DecisionTreeClassifier()),
    'randForestClas':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gradBoostingClas':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [12]:
pipelines.keys()

dict_keys(['logReg', 'rigClas', 'sgdClas', 'svc', 'gaussNb', 'xgbClas', 'desTreeClas', 'randForestClas', 'gradBoostingClas'])

In [13]:
list(pipelines.values())

[Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(solver='liblinear'))]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('sgdclassifier', SGDClassifier())]),
 Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gaussiannb', GaussianNB())]),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('xgbclassifier',
                  XGBClassifier(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, gamma=None, gpu_id=None,
                                importance_type='gain',
                                interaction_constraints=None, learning_rate=None,
           

In [14]:
# train/fit each of the model and place them with their respective algorithm and append all in a dictionary/fit_models
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    #append each model with their respective algorithm to the fit_models
    fit_models[algo]=model

c:\program files\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:23:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [16]:
fit_models['logReg'].predict(x_test)

array(['Happy', 'Happy', 'Happy', 'Victorious', 'Neutral', 'Victorious',
       'Victorious', 'Wakanda Forever', 'Happy', 'Wakanda Forever',
       'Happy', 'Wakanda Forever', 'Neutral', 'Sad', 'Neutral',
       'Victorious', 'Happy', 'Victorious', 'Neutral', 'Victorious',
       'Wakanda Forever', 'Happy', 'Sad', 'Wakanda Forever', 'Victorious',
       'Happy', 'Sad', 'Neutral', 'Wakanda Forever', 'Neutral', 'Happy',
       'Happy', 'Wakanda Forever', 'Neutral', 'Victorious',
       'Wakanda Forever', 'Happy', 'Victorious', 'Happy', 'Happy',
       'Wakanda Forever', 'Victorious', 'Wakanda Forever', 'Sad',
       'Victorious', 'Wakanda Forever', 'Neutral', 'Neutral', 'Neutral',
       'Victorious', 'Happy', 'Neutral', 'Victorious', 'Neutral',
       'Neutral', 'Wakanda Forever', 'Wakanda Forever', 'Happy',
       'Neutral', 'Victorious', 'Sad', 'Victorious', 'Happy', 'Happy',
       'Victorious', 'Wakanda Forever', 'Wakanda Forever', 'Sad', 'Sad',
       'Victorious', 'Wakanda Forever

# Evaluate and Serialize the Models

In [17]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print('Model name: {}'.format(algo))
    print(classification_report(y_test, yhat))

Model name: logReg
                 precision    recall  f1-score   support

          Happy       1.00      0.99      0.99        68
        Neutral       0.98      1.00      0.99        63
            Sad       1.00      1.00      1.00        50
     Victorious       1.00      1.00      1.00        62
Wakanda Forever       1.00      1.00      1.00        58

       accuracy                           1.00       301
      macro avg       1.00      1.00      1.00       301
   weighted avg       1.00      1.00      1.00       301

Model name: rigClas
                 precision    recall  f1-score   support

          Happy       1.00      1.00      1.00        68
        Neutral       1.00      1.00      1.00        63
            Sad       1.00      1.00      1.00        50
     Victorious       1.00      1.00      1.00        62
Wakanda Forever       1.00      1.00      1.00        58

       accuracy                           1.00       301
      macro avg       1.00      1.00      1.

In [18]:
# store the best model(Random Forest Classifier) to disk
with open('facePosehand_Model.pkl', 'wb') as f:
    pickle.dump(fit_models['randForestClas'], f)

# Load and Make Predictions with the Model

In [10]:
with open('facePosehand_Model.pkl', 'rb') as f:
    model = pickle.load(f)

In [11]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [12]:
# initialize the webcam
webcam = VideoStream(src=0).start()
time.sleep(2.0)

holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while True:
    frame = webcam.read()
    frame = imutils.resize(frame, width=800)
    #change from BGR to RGB
    RGB_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    #set the flags to False to prevent coping the image data
    RGB_frame.flags.writeable = False
    
    #printing out the results of the detections
    results = holistic.process(RGB_frame)
    #print(results.face_landmarks)
            
    #set this back to True to copy the image data for rendering
    RGB_frame.flags.writeable = True
    #convert back to BGR for rendering
    BGR_frame = cv2.cvtColor(RGB_frame, cv2.COLOR_RGB2BGR)
            
    # Draw face landmarks
    mp_drawing.draw_landmarks(BGR_frame, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(100,100,0), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(0,230,180), thickness=1, circle_radius=1))

    # Draw pose detections
    mp_drawing.draw_landmarks(BGR_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(210,0,0), thickness=2, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(0,220,0), thickness=2, circle_radius=2))
    
    # Right Hand
    mp_drawing.draw_landmarks(BGR_frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(43,100,0), thickness=3, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(100,43,0), thickness=2, circle_radius=2))
    
    # Left Hand
    mp_drawing.draw_landmarks(BGR_frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(43,100,0), thickness=3, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(100,43,0), thickness=2, circle_radius=2))
            
    try:
        # Extract the pose landmarks
        # store the pose landmarks into a variable pose
        pose = results.pose_landmarks.landmark

        # grab these coordinates(x, y, z & visibility) from pose
        # flatten is used to place all coordinates in a single row
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        # store the pose landmarks into a variable face
        face = results.face_landmarks.landmark

        # grab these coordinates(x, y, z & visibility) from face
        # flatten is used to place all coordinates in a single row
        face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
        
        # concate the pose, face rows and hand rows
        row = pose_row+face_row
        
        # Make Predictions
        target = pd.DataFrame([row])
        predict_target_class = model.predict(target)[0]
        predict_target_prob = model.predict_proba(target)[0]
        #print(predict_target_class, predict_target_prob)
            
        # grab the EAR coordinates of the x and y, and then multiply it by 840x580
        # for our webcam to stream it, we need it in integer and then put it in a tuple
        ear_coords = tuple(np.multiply(
            np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                      results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)),
            [840,580]).astype(np.int))
            
        cv2.rectangle(BGR_frame, (ear_coords[0], ear_coords[1]+5),
                     (ear_coords[0]+len(predict_target_class)*18, ear_coords[1]-25),
                     (212,135,67), -1)
        cv2.putText(BGR_frame, predict_target_class, ear_coords,
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        # get status box
        cv2.rectangle(BGR_frame, (0,0),(250, 60), (212,135,67), -1)

        # Display Class labels
        cv2.putText(BGR_frame, 'LABEL', (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(BGR_frame, predict_target_class.split(' ')[0],
                    (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

        # Display Probability
        cv2.putText(BGR_frame, 'PROB', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(BGR_frame, str(round(predict_target_prob[np.argmax(predict_target_prob)], 2)),
                    (10,40),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    except:
        pass

    cv2.imshow('LIVE!', BGR_frame)
    key = cv2.waitKey(5)
    if key == 27:
        break
        
webcam.stop()
cv2.destroyAllWindows()